In [1]:
from __future__ import annotations

import numpy as np
import pandas as pd
from dataclasses import dataclass
from typing import List, Tuple, Dict, Optional

from scipy.signal import welch
from scipy import stats

from datetime import timedelta, datetime as dt

@dataclass
class SeizureEvent:
    seizure_id: int
    t0: float  # seizure start (s fra recording start)
    t1: float  # seizure end   (s fra recording start)


In [2]:
from src.hrv_epatch.io.tdms import extract_tdms_channel

In [3]:
from src.hrv_epatch.io.data_loader import Load_full_ecg_data

data = Load_full_ecg_data("Patient 5")
patient_id = data["PatientID"]
ecg_df = data["ECG"]
seizure_df = data["Seizures"]
sample_rate = data["SampleRate"]

print(f"Loaded data for {patient_id}:")
print(f"- ECG signal shape: {ecg_df.shape}")
print(f"- Number of seizures: {seizure_df.shape[0]}")
print(f"- Sample rate: {sample_rate} Hz")

print(seizure_df.head())


c:\Users\kvjkv\Git1\Epilepsy_MasterThesis\src\hrv_epatch\io\data_loader.py:83: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.0' has dtype incompatible with datetime64[us], please explicitly cast to a compatible dtype first.
  info.loc[prop, "Value"] = val


NameError: name '_to_local_naive' is not defined

In [17]:
ecg_df["Timestamp"].iloc[0]

Timestamp('2016-10-12 09:05:02')

In [ ]:
# Eksempel: byg liste af SeizureEvent fra din annoterings-DataFrame
# Tilpas kolonnenavne, så de matcher din egen df
seizure_events: List[seizure_df["Anfald nr."]] = []
start_s = ecg_df["Timestamp"].iloc[0]  # antag at dette er starttidspunktet for optagelsen

for _, row in seizure_df.iterrows():
    start_klinisk = pd.to_datetime(f"{row['Dato']} {row['Anfaldsstart Klinisk (tt:mm:ss)']}", format="%d.%m.%y %H:%M:%S") if pd.notna(row['Anfaldsstart Klinisk (tt:mm:ss)']) else None
    stop_klinisk = pd.to_datetime(f"{row['Dato']} {row['Anfaldstop Klinisk (tt:mm:ss)']}", format="%d.%m.%y %H:%M:%S") if pd.notna(row['Anfaldstop Klinisk (tt:mm:ss)']) else None
    start_eeg = pd.to_datetime(f"{row['Dato']} {row['Anfaldsstart EEG (tt:mm:ss)']}", format="%d.%m.%y %H:%M:%S") if pd.notna(row['Anfaldsstart EEG (tt:mm:ss)']) else None
    stop_eeg = pd.to_datetime(f"{row['Dato']} {row['Anfaldstop EEG (tt:mm:ss)']}", format="%d.%m.%y %H:%M:%S") if pd.notna(row['Anfaldstop EEG (tt:mm:ss)']) else None

    if start_klinisk and stop_klinisk:
        klinisk_duration = (stop_klinisk - start_klinisk).total_seconds()
    else:
        klinisk_duration = 0

    if start_eeg and stop_eeg:
        eeg_duration = (stop_eeg - start_eeg).total_seconds()
    else:
        eeg_duration = 0

    # print(f"Klinisk duration: {klinisk_duration} seconds")
    # print(f"EEG duration: {eeg_duration} seconds")
    # print("----")
    
    if klinisk_duration >= eeg_duration and start_klinisk:
        t0 = (start_klinisk - start_s).total_seconds()
        t1 = t0 + klinisk_duration
    elif start_eeg:
        t0 = (start_eeg - start_s).total_seconds()
        t1 = t0 + eeg_duration
    else:
        continue  # Skip this row if no valid annotation is present

    ev = SeizureEvent(
        seizure_id=int(row["Anfald nr."]),
        t0=t0,
        t1=t1,
    )
    seizure_events.append(ev)

len(seizure_events), seizure_events[:3]


Klinisk duration: 120.0 seconds
EEG duration: 27.0 seconds
----
Klinisk duration: 93.0 seconds
EEG duration: 115.0 seconds
----
Klinisk duration: 130.0 seconds
EEG duration: 128.0 seconds
----


(3,
 [SeizureEvent(seizure_id=1, t0=80515.0, t1=80635.0),
  SeizureEvent(seizure_id=2, t0=106947.0, t1=107062.0),
  SeizureEvent(seizure_id=3, t0=109886.0, t1=110016.0)])

In [8]:
from src.hrv_epatch.io.tdms_pipeline import load_tdms, align_annotations_to_samples, read_annotations
path = r"E:\ML algoritme tl anfaldsdetektion vha HRV\ePatch data from Aarhus to Lausanne\Patients ePatch data\Patient 5\recording 1\Patient 5_1.tdms"
signal, meta = load_tdms(path)

print("fs:", meta.fs)
print("start_time:", meta.start_time, "tzinfo:", getattr(meta.start_time, "tzinfo", None))
# For naive-local stil: tzinfo skal være None og klokkeslæt skal være "lokal" (fx 11:05:02)

path_ann = r"E:\ML algoritme tl anfaldsdetektion vha HRV\ePatch data from Aarhus to Lausanne\Seizure log ePatch patients with seizures - excluded seizures removed\Patient 5.xls"
ann = read_annotations(path_ann)
ann_idx = align_annotations_to_samples(ann, meta)
print(ann_idx[["onset_time","offset_time"]].head())  # bør være naive datetimes (ingen +02:00)


fs: 512.0
start_time: 2016-10-12 11:05:02 tzinfo: None


ValueError: Could not locate a 'start/onset' column in annotations file. Columns found: ['unnamed: 0', 'unnamed: 1', 'unnamed: 2', 'unnamed: 3', 'unnamed: 4', 'unnamed: 5', 'unnamed: 6', 'unnamed: 7', 'unnamed: 8', 'label']